In [0]:
import os
import numpy as np
import tensorflow as tf
import numpy as np
import cv2
import random
import itertools
import pickle

In [2]:
!wget http://mla.sdu.edu.cn/sjk/Finger%20Vein%20Database.rar
!unrar x -r "Finger Vein Database.rar"
!mv "Finger Vein Database" "Finger"
  

--2019-03-30 05:26:19--  http://mla.sdu.edu.cn/sjk/Finger%20Vein%20Database.rar
Resolving mla.sdu.edu.cn (mla.sdu.edu.cn)... 202.194.15.5, 2001:da8:7000:15:202:194:15:5
Connecting to mla.sdu.edu.cn (mla.sdu.edu.cn)|202.194.15.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132774451 (127M) [application/x-rar-compressed]
Saving to: ‘Finger Vein Database.rar’

Finger Vein Databas   2%[                    ]   3.09M  24.7KB/s    eta 36m 37s^C

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Finger Vein Database.rar

Creating    Finger Vein Database                                      OK
Creating    Finger Vein Database/001                                  OK
Creating    Finger Vein Database/001/left                             OK
Extracting  Finger Vein Database/001/left/index_1.bmp                      1%  OK 
Extracting  Finger Vein Database/001/left/index_2.bmp                      2%  OK 
Extracting  Fing

In [0]:
DATADIR="Finger"
IMG_SIZE=200
def load_image(addr):
    img = cv2.imread(addr,cv2.IMREAD_GRAYSCALE)
    # OTSU THRESHOLDING
    # img2gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    res = cv2.bitwise_and(img,img,mask = mask)
    
    # cv2.imshow("image",res)
    # cv2.imshow("image2",img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    new = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
    return new

traning_data=[]
# for person in os. listdir(DATADIR):
#     path=os.path.join(DATADIR,person)
#     for hand in os.listdir(path):
#         path_=os.path.join(path,hand)
#         for finger in os.listdir(path_):
#             if(finger[-3:]=='bmp'):
#                 img_path=os.path.join(path_,finger)
#                 img  = load_image(img_path)
#                 label = person+"_"+hand+"_"+finger[:-6]
#                 traning_data.append((img,label))
    
#Add Positive and Anchor
for person in os. listdir(DATADIR):
    path=os.path.join(DATADIR,person)
    for hand in os.listdir(path):
        path_=os.path.join(path,hand)
        temp=[]
        previous_finger=None
        for finger in os.listdir(path_):
            if(finger[-3:]=="bmp"):
                if(finger[:4]==previous_finger):
                    img_path = os.path.join(path_,finger)
                    label = person+"_"+hand+"_"+finger[:-6]
                    temp.append([load_image(img_path),label])
                else:
                    previous_finger = finger[:4]
                    for t in itertools.combinations(temp,2):
                        traning_data.append(list(t))
                    img_path = os.path.join(path_,finger)
                    label = person+"_"+hand+"_"+finger[:-6]
                    temp = [[load_image(img_path),label]]
print("Length of Training Data: "+str(len(traning_data)))

del person
del hand
del finger
#Add a negative
for t in range(0,len(traning_data)):
    while True:
        i = random.randint(0,len(traning_data)-1)
        if(traning_data[i][i%2][1]!=traning_data[t][0][1]):
            traning_data[t].append(traning_data[i][i%2])
            break

A=[]
B=[]
C=[]
# print(traning_data[0])
# for tool in traning_data:
#     A.append(tool[0][0])
#     B.append(tool[1][0])
#     C.append(tool[2][0])
for x in range(0,len(traning_data)):
    A.append(traning_data[x][0][0])
    B.append(traning_data[x][1][0])
    C.append(traning_data[x][2][0])
del traning_data
del x
#         X.append(features)
#         # Y.append(label)
# random.shuffle(X)
# print(X[0])
# for i,j,k in X:
#     A.append(i)
#     B.append(j)
#     C.append(k)
# print(A)
A = np.array(A).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
B = np.array(B).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
C = np.array(C).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
A=A/255.0
B = B/255.0
C = C/255.0
# pickle_out = open("A.pickle","wb")
# pickle.dump(A, pickle_out)
# pickle_out.close()

# pickle_out = open("B.pickle","wb")
# pickle.dump(B, pickle_out)
# pickle_out.close()

# pickle_out = open("C.pickle","wb")
# pickle.dump(C, pickle_out)
# pickle_out.close()

# print("Pickle File Generated")



Length of Training Data: 2332


NameError: ignored

In [0]:
print(dataset_A)

<DatasetV1Adapter shapes: (200, 200, 1), types: tf.float64>


In [0]:
learning_rate = 0.0001
epochs = 10
batch_size = 50

checkpoint_path=os.getcwd()
print(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_weights_only=True,verbose=1)

# pickle_in = open("A.pickle","rb")
# A = pickle.load(pickle_in)

# pickle_in = open("B.pickle","rb")
# B = pickle.load(pickle_in)

# pickle_in = open("C.pickle","rb")
# C = pickle.load(pickle_in)

def CNN_layer(x,seq_no):
    x= tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding = "valid", name=seq_no+"conv1", activation="relu" , input_shape= x.shape[1:])(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (1, 1), padding = "valid", name=seq_no+"pool1")(x)

    x= tf.keras.layers.Conv2D(64, (1, 1), strides=(1, 1), padding = "valid", name=seq_no+"conv2a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(192, (3, 3), strides=(1, 1), padding = "valid", name=seq_no+"conv2", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name=seq_no+"pool2")(x)


    x= tf.keras.layers.Conv2D(192, (1, 1), strides=(1, 1), padding = "valid", name=seq_no+"conv3a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(384, (3, 3), strides=(2, 2), padding = "valid", name=seq_no+"conv3", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name=seq_no+"pool3")(x)

    x= tf.keras.layers.Conv2D(384, (1, 1), strides=(1, 1), padding = "valid", name=seq_no+"conv4a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name=seq_no+"conv4", activation="relu")(x)

    x= tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding = "valid", name=seq_no+"conv5a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name=seq_no+"conv5", activation="relu")(x)

    x= tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), padding = "valid", name=seq_no+"conv6a", activation="relu")(x)
    x= tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), padding = "valid", name=seq_no+"conv6", activation="relu")(x)
    x= tf.keras.layers.MaxPooling2D(pool_size= (3, 3), strides = (2, 2), padding = "valid", name=seq_no+"pool4")(x)
    return x
# dropout
input_=[]
densors = []

input_.append(tf.keras.layers.Input(shape=A.shape[1:]))
input_.append(tf.keras.layers.Input(shape=B.shape[1:]))
input_.append(tf.keras.layers.Input(shape=C.shape[1:]))

count=0
for curr_input in input_:
    densors.append(CNN_layer(curr_input,str(count)))
    count=count+1

y = tf.keras.layers.Concatenate()(densors)

y = tf.keras.layers.Flatten()(y)
y = tf.keras.layers.Dense(513)(y)

split = tf.keras.layers.Lambda( lambda x: tf.split(x,num_or_size_splits=3,axis=1))(y)
densors=[]
for i in range(3):
    densors.append(tf.keras.layers.Dense(1)(split[0]))
y = tf.keras.layers.Concatenate()(densors)
y = tf.keras.layers.Activation("sigmoid")(y)
model=tf.keras.models.Model(inputs=input_,outputs=y)
model.compile("adam", loss='binary_crossentropy')


/content
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# model.fit([A,B,C], batch_size=16, epochs=10, steps_per_epoch=16, validation_split=0.3, verbose=2)
# model.save_weights(checkpoint_path)
# model.summary()

In [0]:
loss = tf.contrib.losses.metric_learning.triplet_semihard_loss()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())

mod = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="train/mnist_convnet_model")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_scaled},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)


In [0]:
model.train_on_batch([A,B,C])

Instructions for updating:
Use tf.cast instead.
